In [1]:
from carla import Benchmark
from IPython.display import display
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
scm = CausalModel("credit")
dataset = scm.generate_dataset(1000)
dataset.df.to_csv('credit_synthetic.csv',index=False)
display(dataset.df)

,label,x1,x2,x3,x4,x6,x5,x7
0,0.0,1.0,-2.901628,0.069036,3.422301,1.637961,0.593002,-2.128538
1,1.0,0.0,7.152467,-0.081706,-0.055258,-1.595548,0.480908,2.487011
2,1.0,0.0,4.102632,-0.206645,2.465405,2.844562,5.338350,-5.896902
3,1.0,1.0,7.374361,-0.134668,0.112714,1.412981,1.394632,-7.577527
4,0.0,1.0,2.021415,0.030820,4.122077,0.107332,3.838096,-2.995881
...,...,...,...,...,...,...,...,...
995,0.0,0.0,-9.753303,-0.018339,1.108545,-3.506350,-0.588439,0.758632
996,0.0,1.0,-3.061453,0.119317,0.613308,-0.231908,1.068151,-8.115625
997,1.0,1.0,3.365306,0.083312,-0.925692,8.535162,2.829978,10.365376
998,0.0,0.0,-13.266146,-0.004838,-2.754319,-5.475563,0.167984,-3.749375


In [3]:
'''Build Dataset for Wachter'''
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./credit_synthetic.csv')
continuous = dataframe.drop(columns=['label']).columns

dataset = CsvCatalog(file_path="credit_synthetic.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='label')

In [4]:
#Model 

training_params = {"lr": 0.01, "epochs": 3, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.452, balance on test set 0.504
Epoch 0/2
----------
train Loss: 0.6853 Acc: 0.5760

test Loss: 0.6499 Acc: 0.7040

Epoch 1/2
----------
train Loss: 0.5656 Acc: 0.7293

test Loss: 0.5095 Acc: 0.8040

Epoch 2/2
----------
train Loss: 0.4747 Acc: 0.8093

test Loss: 0.5793 Acc: 0.7000



In [5]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,label,x1,x2,x3,x4,x6,x5,x7
7,0.0,1.0,0.150878,0.257247,0.564962,0.586630,0.193796,0.791498
12,0.0,0.0,0.139977,0.473798,0.703149,0.506172,0.160075,0.486864
19,0.0,0.0,0.236518,0.330420,0.769651,0.431542,0.547872,0.192275
30,0.0,1.0,0.098774,0.736847,0.601899,0.277447,0.479366,0.553057
32,0.0,0.0,0.223250,0.283348,0.656265,0.211635,0.478846,0.547856
33,0.0,0.0,0.186938,0.324934,0.744256,0.423261,0.410593,0.508765
36,0.0,0.0,0.235098,0.441379,0.662188,0.308735,0.223921,0.339314
37,0.0,0.0,0.223860,0.650618,0.618316,0.000000,0.490678,0.115417
40,0.0,0.0,0.207427,0.195322,0.713388,0.445506,0.510982,0.578401
48,0.0,1.0,0.183681,0.383227,0.851397,0.572500,0.460002,0.400448


In [6]:
hyperparams = {"loss_type": "BCE"}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
cfs = recourse_method.get_counterfactuals(test_factual)
#from carla.models.negative_instances import predict_negative_instances
#from carla.recourse_methods.catalog.causal_recourse import (
#    CausalRecourse,
#    constraints,
#    samplers,
#)

#hyperparams = {
#    "optimization_approach": "brute_force",
#    "num_samples": 10,
#    "scm": scm,
#    "constraint_handle": constraints.point_constraint,
#    "sampler_handle": samplers.sample_true_m0,
#}
#cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

,x1,x2,x3,x4,x5,x6,x7
7,1.010000,0.160878,0.267247,0.554962,0.203796,0.596630,0.801498
12,0.088494,0.228526,0.562308,0.614642,0.248615,0.594692,0.575398
19,0.038755,0.276136,0.369862,0.730123,0.587473,0.471266,0.232008
30,0.975852,0.128665,0.766715,0.572029,0.509253,0.307331,0.582945
32,0.019950,0.243207,0.303300,0.636314,0.498802,0.231588,0.567812
33,0.039692,0.226655,0.364634,0.704558,0.450306,0.462965,0.548475
36,0.078511,0.313666,0.519907,0.583664,0.302480,0.387273,0.417866
37,0.013175,0.313035,0.738475,0.530253,0.579792,0.082588,0.202939
40,0.010000,0.217427,0.205322,0.703388,0.520982,0.455506,0.588401
48,0.983039,0.203645,0.403180,0.831444,0.479964,0.592461,0.420410


In [7]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x7'][0]
    return labels

In [12]:
mapping_dict={ 
    # Gender
      'u1': 'x1',
      # Age
      'u2': 'x2',
      # Education
      'u3': 'x3',
      # Loan amount
      'u4':'x4',
      # Loan duration
      'u5': 'x5',
      # Income
      'u6': 'x6',
      # Savings
      'u7':'x7',
}

In [13]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass


metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


0


'Histogram Features / true Positive / false Positive'